# 결정 트리 실습

사용자 행동 인식 데이터 세트

[원본 데이터셋]
https://archive.ics.uci.edu/ml/datasets/human+activity+recognition+using+smartphones

데이터 세트 정보:

실험은 19-48세 사이의 30명의 지원자 그룹으로 수행되었습니다. 각자 스마트폰(Samsung Galaxy S II)을 허리에 차고 6가지 활동(
1:WALKING, 
2:WALKING_UPSTAIRS, 
3:WALKING_DOWNSTAIRS, 4:SITTING, 5:STANDING, 6:LAYING)을 수행했습니다. 내장된 가속도계와 자이로스코프를 사용하여 50Hz의 일정한 속도로 3축 선형 가속도와 3축 각속도를 캡처했습니다. 실험은 데이터에 수동으로 레이블을 지정하기 위해 비디오로 녹화되었습니다. 얻은 데이터 세트는 훈련 데이터 생성을 위해 70%, 테스트 데이터 생성을 위해 30%가 선택된 두 세트로 무작위로 분할되었습니다.

센서 신호(가속도계 및 자이로스코프)는 노이즈 필터를 적용하여 사전 처리된 다음 2.56초 및 50% 중첩(128 판독/창)의 고정 너비 슬라이딩 창에서 샘플링되었습니다. 중력 및 신체 운동 성분을 갖는 센서 가속도 신호는 Butterworth 저역 통과 필터를 사용하여 신체 가속도와 중력으로 분리되었습니다. 중력은 저주파 성분만 있다고 가정하므로 차단 주파수가 0.3Hz인 필터를 사용했습니다. 각 창에서 시간 및 주파수 영역에서 변수를 계산하여 특징 벡터를 얻었습니다.

이 데이터 세트에 대한 자세한 내용은 README.txt 파일을 확인하십시오.

참가자 중 한 명과 함께 녹화된 6가지 활동의 예를 포함하는 실험 비디오는 다음 링크에서 볼 수 있습니다.

https://www.youtube.com/watch?v=XOEN9W05_4A]


**Mission**

수집된 데이터를 기반으로 결정트리를 이용해 어떤 행동을 하고 있는지 예측해 봅시다.

In [ ]:
# ## 코랩을 사용할 때
# #드라이브 마운트
# from google.colab import drive
# drive.mount('/content/drive')

# #현재 작업 위치 이동
# #띄어쓰기에 \붙일 것
# %cd /content/drive/Othercomputers/내\ 노트북_before/Devpy/13.머신러닝

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

## 데이터 로드

1. 피처만 있는 파일을 로드
2. 훈련데이터와 피처를 합쳐 훈련 데이터 프레임 만듬
3. 데스트데이터와 피처를 합쳐 테스트 데이터 프레임 만듬

In [2]:
# 피처이름 로드 
feature_name_df = pd.read_csv('./dataset/human_activity/features_new.txt', header=None,names=['column_name'])


In [3]:
# 피처이름을 리스트로
feature_name = feature_name_df['column_name'].values.tolist()

In [4]:
# 학습 피처 데이터 셋과 테스트 피처 데이터을 DataFrame으로 로딩. 컬럼명은 feature_name 적용
X_train = pd.read_csv('./dataset/human_activity/train/X_train.txt',sep='\s+', names=feature_name )
X_test = pd.read_csv('./dataset/human_activity/test/X_test.txt',sep='\s+', names=feature_name)

In [5]:
# 학습 레이블과 테스트 레이블 데이터을 DataFrame으로 로딩하고 컬럼명은 action으로 부여
y_train = pd.read_csv('./dataset/human_activity/train/y_train.txt',sep='\s+',header=None,names=['action'])
y_test = pd.read_csv('./dataset/human_activity/test/y_test.txt',sep='\s+',header=None,names=['action'])

In [6]:
#action 피처의 class별 갯수
y_train['action'].value_counts
y_test['action'].value_counts

<bound method IndexOpsMixin.value_counts of 0       5
1       5
2       5
3       5
4       5
       ..
2942    2
2943    2
2944    2
2945    2
2946    2
Name: action, Length: 2947, dtype: int64>

In [8]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# DesisionTree분류 클래스 사용(randomstate 156)
dt_clf = DecisionTreeClassifier(random_state=156)
# 훈련
dt_clf.fit(X_train, y_train)

# 예측
pred = dt_clf.predict(X_test)

#평가
accuracy_score(y_test, pred)

0.8547675602307431

In [9]:
# DecisionTreeClassifier의 하이퍼 파라미터 추출
print('DecisionTreeClassifier 기본 하이퍼 파라미터:\n', dt_clf.get_params())

DecisionTreeClassifier 기본 하이퍼 파라미터:
 {'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'random_state': 156, 'splitter': 'best'}


## 하이퍼파라미터 튜닝1

In [10]:
from unittest.mock import MagicProxy
from sklearn.model_selection import GridSearchCV

#max_depth를 바꿔가며 훈련해보자
#나무의 깊이를 얼마만큼 성장시킬것인가

params = {
    'max_depth' : [ 6, 8 ,10, 12, 16 ,20, 24]
}


In [11]:
# GridSearchCV 사용
grid_clf = GridSearchCV(dt_clf, param_grid=params, scoring='accuracy', cv=3)


# 훈련
grid_clf.fit(X_train, y_train)


GridSearchCV(cv=3, estimator=DecisionTreeClassifier(random_state=156),
             param_grid={'max_depth': [6, 8, 10, 12, 16, 20, 24]},
             scoring='accuracy')

In [12]:
# GridSearchCV객체의 cv_results_ 속성을 DataFrame으로 생성.
grid_clf.cv_results_
cv_results = pd.DataFrame(grid_clf.cv_results_)
cv_results

{'mean_fit_time': array([37.82317932,  9.28864733, 10.48305527, 12.24272005, 13.11912735,
        12.65613707, 13.74826709]),
 'std_fit_time': array([22.80941713,  0.80887164,  0.63364977,  0.75024332,  1.10750545,
         0.41482072,  1.77331744]),
 'mean_score_time': array([0.55867473, 0.04592594, 0.04387474, 0.05729167, 0.04585274,
        0.04226454, 0.05281734]),
 'std_score_time': array([0.65458304, 0.00169436, 0.00308727, 0.01509993, 0.00110717,
        0.0036304 , 0.00414775]),
 'param_max_depth': masked_array(data=[6, 8, 10, 12, 16, 20, 24],
              mask=[False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'max_depth': 6},
  {'max_depth': 8},
  {'max_depth': 10},
  {'max_depth': 12},
  {'max_depth': 16},
  {'max_depth': 20},
  {'max_depth': 24}],
 'split0_test_score': array([0.83924929, 0.82945736, 0.83190534, 0.83843329, 0.82252142,
        0.83312933, 0.83312933]),
 'split1_test_score': array([0.86821705, 

In [16]:
#베스트 분류기 선택
print('최적 하이퍼 파라미터:{0}, 최적 평균 정확도:{1:.3f}'.format(grid_clf.best_params_, 
                                                  grid_clf.best_score_))
best_clf = grid_clf.best_estimator_

 #예측
pred = best_clf.predict(X_test)

최적 하이퍼 파라미터:{'max_depth': 6}, 최적 평균 정확도:0.847


In [37]:
#평가
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score 
from sklearn.metrics import accuracy_score, precision_score , recall_score , confusion_matrix

def get_clf_eval(y_test, pred):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred, average='micro')
    recall = recall_score(y_test, pred, average='micro')
    # F1 스코어 추가
    f1 = f1_score(y_test,pred, average='micro')
    print('오차 행렬')
    print(confusion)
    # f1 score print 추가
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}, F1:{3:.4f}'.format(accuracy, precision, recall, f1))

In [38]:
get_clf_eval(y_test, pred)

오차 행렬
[[461   5  30   0   0   0]
 [ 43 343  85   0   0   0]
 [ 27  45 348   0   0   0]
 [  0   0   0 344 147   0]
 [  0   0   0  43 489   0]
 [  0   0   0   0   0 537]]
정확도: 0.8558, 정밀도: 0.8558, 재현율: 0.8558, F1:0.8558


ValueError: average가 binary로 설정되어있다고 다른 걸로 선택하라는 에러가 뜸. 처음에는 precision쪽에 에러가 떠서 추가했더니 recall에도 에러떠서 averag파라미터 설정을 해주니까 갑자기 해결됨. 

> * average: 다중 분류 모델에서 사용하는 파라미터. 디폴트는 binary. [None,micro,macro,weighted,samples,binary] 중 선택. 

이번 실습때는 무엇 때문에 이 에러가 발생한건지 의문.. 

```
    하이퍼파라미터 튜닝으로 성능이 향상 되었나요?
```

84.7% -> 85.5% 로 근소하게 성능 향상됨

## 하이퍼파라미터 튜닝2

In [39]:
#하이퍼파라미터
params = {
    'max_depth' : [ 8 , 12, 16 ,20], 
    'min_samples_split' : [16,24],
}


In [24]:
# GridSearchCV 사용
grid_clf = GridSearchCV(dt_clf, param_grid=params, scoring='accuracy', cv=3)

# 훈련
grid_clf.fit(X_train, y_train)


GridSearchCV(cv=3, estimator=DecisionTreeClassifier(random_state=156),
             param_grid={'max_depth': [8, 12, 16, 20],
                         'min_samples_split': [16, 24]},
             scoring='accuracy')

In [40]:
#베스트 분류기 선택
best_clf = grid_clf.best_estimator_


#예측
pred = best_clf.predict(X_test)


#평가
get_clf_eval(y_test, pred)

오차 행렬
[[471   9  16   0   0   0]
 [ 75 365  31   0   0   0]
 [ 13  57 350   0   0   0]
 [  0   0   0 371 120   0]
 [  0   0   0  57 475   0]
 [  0   0   0   0   0 537]]
정확도: 0.8717, 정밀도: 0.8717, 재현율: 0.8717, F1:0.8717


```
    하이퍼파라미터 튜닝으로 성능이 향상 되었나요?
```

85.5% -> 87.1% 로 1.6%p 성능이 향상됨. 첫번째 튜닝 결과(0.8%p)의 두 배    

---

**[생활탐구]**


```
방문을 꼭 닫은채 공부하고 있다는 보검이...
정말 보검이는 공부하고 있을까요?
생체 신호 측정기를 착용한 보검이로 부터 아래와 같은 신호를 가져왔습니다. 
지금 현재 보검이는 무엇을 하고 있나요?

[보검이 생체 신호 파일]X_quiz.txt 

```

In [41]:
#신호 읽어서 예측해보기
bogum = pd.read_csv('./dataset/human_activity/test/X_quiz.txt', header=None)

In [42]:
pred = best_clf.predict(bogum)
pred

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


array([6])

6. Laying 
보검이는 누워있음